# Part 2 - The hidden layer

Welcome to Part 2!

Here, we'll implement a neural network with 2 layers (a hidden and output layer), and talk about feedforward and backpropagation. We'll take advantage of Numpy, a library that provides fast alternatives to math operations in Python and is designed to work efficiently with groups of numbers - like matrices.

## Index

- [Multi-layer perceptron](#multi-layer-perceptron)
- [The weights](#the-weigts)
- [Feedforward](#feedforward)
  - The hidden layer
  - The output layer
- Backpropagation
  - Caculating the error
  - Learning
- Full implementation in a real case

## Multi-layer perceptron <a id='multi-layer-perceptron'></a>

In [Part 1](Part1.ipynb) we implemented the simplest neural network - a perceptron. This neural network doesn't have a hidden layer so it can't help us to find predictions to complex problems.

When we combine perceptrons so that the output of one becomes the input of another one, we form a multi-layer perceptron or a neural network.

Neural networks have a certain special architecture with layers:

![Neural network](img/part2/1.png)

- **Input layer:** contains the inputs $x_1$, $x_2$, $...$, $x_n$, $1$.
- **Hidden layer:** set of linear models created with the first input layer.
- **Output layer:** where the linear models get combined to obtain a nonlinear model.

Now, not all neural networks look like the one above. They can be way more complicated. In particular, we can do the following things:
- Add more nodes to the input, hidden and output layers.
- Add more layers.

The following image shows the network with which we will work, with its input units, labeled $x_1$, $x_2$, and $x_3$, its hidden nodes labeled $h_1$ and $h_2$, and all of the weights between the input layer and the hidden layer, labeled with their appropriate $w_{ij}$ indices:

![Neural network for this exercise](img/part2/2.png)

## The weigts <a id='the-weigts'></a>

The weights need to be stored in a **matrix**, indexed as $w_{ij}$. Each **row** in the matrix will correspond to the weights **leading out** of a **single input unit**, and each **column** will correspond to the weights **leading in** to a **single hidden unit**.

For our three input units and two hidden units, the weights matrix looks like this:

![Weights matrix](img/part2/3.png)

To initialize these weights in Numpy, we have to provide the shape of the matrix. If `features` is a 2D array containing the input data:

In [1]:
import numpy as np

# Use to same seed to make debugging easier
np.random.seed(21)

features = np.array([
    [1, 2, 3], 
    [4, 5, 6]])

# Number of records and input units
n_records, n_inputs = features.shape

# Input to hidden weights
n_hidden = 2
weights_input_to_hidden = np.random.normal(0, n_inputs**-0.5, size=(n_inputs, n_hidden))

# Hidden to output weights
weights_hidden_to_output = np.random.normal(0, n_inputs**-0.5, size=n_hidden)

This creates a 2D array (i.e. matrix) named `weights_input_to_hidden` with dimensions `n_inputs` by `n_hidden`:

In [2]:
print(weights_input_to_hidden)

[[-0.03000157 -0.06419907]
 [ 0.60148166 -0.72557877]
 [ 0.43034978 -0.98787735]]


and a 2D array named `weights_hidden_to_output` with dimensions `n_hidden`:

In [3]:
print(weights_hidden_to_output)

[-0.11885586 -0.1354298 ]


## Feedforward <a id='feedforward'></a>

Feedforward is the process neural networks use to turn the input into an output:

![Feedforward](img/part2/4.png)

On a multi-layer perceptron or neural network, to calculate a prediction $\hat {y}\,$ we start with the unit vector $x$ and then we apply the first matrix $W^{(1)}$ and a sigmoid activation function to get the values in the second layer. Then we apply the second matrix $W^{(2)}$ and another sigmoid function to get the values on the third layer, and so on and so forth, until we get our final prediction $\hat y$. And this is the feedforward process that neural networks use to obtain the prediction from the input vector.

Let's see step by step this process with our neural network with two layers (hidden and output layer).

### The hidden layer

The input to a hidden unit is the sum of all the inputs multiplied by the hidden unit's weights. So for each hidden layer unit, $h_j$, we need to calculate the following:
$$
h_j = \sum_i w_{ij}x_i
$$

To do that, we need to use **matrix multiplication**.

In this case, we're multiplying the inputs (a row vector here) by the weights. To do this, you take the dot (inner) product of the inputs with each column in the weights matrix. For example, to calculate the input to the first hidden unit, $j=1$, you'd take the dot product of the inputs with the first column of the weights matrix, like so:

![Input to the first hidden unit](img/part2/5.png)

$$
h_1 = w_{11}x_1 + w_{21}x_2 + w_{31}x_3
$$

And for the second hidden layer input, you calculate the dot product of the inputs with the second column. And so on and so forth.

In Numpy, you can do this for all the inputs and all the outputs at once using `np.dot`:

In [7]:
input = features[0]

# Calculate the inputs for the hidden layer
hidden_inputs = np.dot(input, weights_input_to_hidden)

print('input')
print(input)
print('\nweights_input_to_hidden')
print(weights_input_to_hidden)
print('\nhidden_inputs')
print(hidden_inputs)

input
[1 2 3]

weights_input_to_hidden
[[-0.03000157 -0.06419907]
 [ 0.60148166 -0.72557877]
 [ 0.43034978 -0.98787735]]

hidden_inputs
[ 2.46401109 -4.47898866]


Now that you have the inputs for the hidden layer, you calculate the outputs of that hidden layer passing the inputs through an activation function, which in this case we use the sigmoid function:

In [8]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

hidden_outputs = sigmoid(hidden_inputs)
hidden_outputs

array([ 0.92158004,  0.01121762])

After this process we have our neural network with values in each of its hidden units:

![Neural network with values in each of its hidden units](img/part2/6.png)

### The output layer

Now that you have the outputs for the hidden layer, it's time to calculate the input for the output unit, and this process is the same as with the hidden layer, but instead of multiplying the inputs by the hidden unit's weights, we multiply the outputs for the hidden layer by the output unit's weights:

In [9]:
output_inputs = np.dot(hidden_outputs, weights_hidden_to_output)

print('hidden_outputs')
print(hidden_outputs)
print('\nweights_hidden_to_output')
print(weights_hidden_to_output)
print('\noutput_inputs')
print(output_inputs)

hidden_outputs
[ 0.92158004  0.01121762]

weights_hidden_to_output
[-0.11885586 -0.1354298 ]

output_inputs
-0.11105438401


And finally, we calculate the sigmoid of the result to have our prediction $\hat y$:

In [10]:
output = sigmoid(output_inputs)
output

0.47226490306194757

![Prediction](img/part2/7.png)

# Feedback